In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys

import matplotlib.pyplot as plt
import IPython.display as ipd

from tqdm import tqdm

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
from torchaudio.datasets import SPEECHCOMMANDS
import os


class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__("./", download=True)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as fileobj:
                return [os.path.normpath(os.path.join(self._path, line.strip())) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]
            # 筛选代表数字的音频文件
        digits = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
        self._walker = [w for w in self._walker if any(digit in w for digit in digits)]
        self.labels = []
    def collect_labels(self):
        # 收集所有唯一的标签
        for _, _, label, _, _ in self:
            if label not in self.labels:
                self.labels.append(label)
        self.labels.sort()

# # Create training and testing split of the data. We do not use validation in this tutorial.
# train_set = SubsetSC("training")
# test_set = SubsetSC("testing")

# waveform, sample_rate, label, speaker_id, utterance_number = train_set[0]

In [6]:
import torch
from torch.utils.data import DataLoader
import torchaudio.transforms as T
from torch.utils.data.dataset import random_split

sample_rate=16000
new_sample_rate = 8000
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)

from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # 初始化特征和标签列表
    inputs, labels = [], []

    # 定义一个临时列表来存储所有波形的长度
    lengths = []

    # 提取波形和标签
    for waveform, _, label, _, _ in batch:
        lengths.append(waveform.size(1))
        inputs.append(waveform.squeeze(0)) 
        labels.append(label)

    # 找到最大的波形长度
    max_len = max(lengths)

    # Pad输入波形到相同的长度
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)

    # 将标签转换为张量
    label_to_index = {label: index for index, label in enumerate(sorted(set(labels)))}
    labels_indices = torch.tensor([label_to_index[label] for label in labels])

    return inputs_padded, labels_indices

# all_data = SubsetSC(subset=None)  # 加载所有数据
# total_size = len(all_data)
# train_size = int(total_size * 0.8)
# test_size = total_size - train_size
# train_dataset, test_dataset = random_split(all_data, [train_size, test_size])

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
# train_dataset, test_dataset = random_split(all_data, [train_size, test_size])

# 使用SubsetSC类和collate_fn来创建DataLoader
if device == "cuda":
    num_workers = 16
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False
train_set = SubsetSC(subset='training')
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_fn, num_workers=num_workers, pin_memory=pin_memory)
val_set = SubsetSC(subset='validation')
val_loader = DataLoader(val_set, batch_size=64, shuffle=True, collate_fn=collate_fn, num_workers=num_workers, pin_memory=pin_memory)
test_set = SubsetSC(subset='testing')
test_loader = DataLoader(test_set, batch_size=64, shuffle=False, collate_fn=collate_fn, num_workers=num_workers, pin_memory=pin_memory)

train_set.collect_labels()
print("Training set labels:", train_set.labels)

Training set labels: ['eight', 'five', 'four', 'nine', 'one', 'seven', 'six', 'three', 'two', 'zero']


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Perceptron(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.fc(x)
        return out


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [30]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)

In [43]:
# 参数设置
input_size =  16000
hidden_size = 64
num_classes = 10 
learning_rate = 0.01
num_epochs = 10
best_accuracy = 0.0
# 模型、损失函数和优化器
model = Perceptron(input_size, num_classes)
# model = MLP(input_size, hidden_size, num_classes)
model.to(device)

print(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
log_interval = 20
n_epoch = 2

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
transform = transform.to(device)
with tqdm(total=num_epochs) as pbar:     
    for epoch in range(1, num_epochs + 1):
        # 训练过程...
        model.train()  # 确保模型处于训练模式
        for batch_idx, (features, labels) in enumerate(train_loader):
            k=len(features)
            labels=labels.to(device)
            features = features.view(features.size(0), -1)
            features=features.to(device)
            outputs = model(features)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print(f"Train Epoch: {epoch} [{batch_idx * k}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

            # update progress bar
            pbar.update(pbar_update)
            # record loss
            losses.append(loss.item())

        # 验证过程
        model.eval()  # 设置模型为评估模式
        correct = 0
        total = 0
        with torch.no_grad():
            for features, labels in val_loader:
                features=features.to(device)
                labels=labels.to(device)
                features = features.view(features.size(0), -1)
                outputs = model(features)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.4f}')
        
        # 检查是否为最佳模型，并保存
        if accuracy > best_accuracy:
            print(f"Found better model at epoch {epoch+1} with accuracy {accuracy:.4f}. Saving model...")
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'best_model_accuracy.pt')  # 保存最佳模型的权重
        scheduler.step()


cuda


  0%|          | 0.0018115942028985507/10 [00:00<20:53, 125.36s/it]

Train Epoch: 1 [0/31158 (0%)]	Loss: 2.310135


  0%|          | 0.038043478260869575/10 [00:04<21:06, 127.13s/it] 

Train Epoch: 1 [1280/31158 (4%)]	Loss: 3.171701


...

Train Epoch: 10 [29440/31158 (94%)]	Loss: 2.905098


 88%|████████▊ | 8.81159420289811/10 [20:58<02:28, 124.60s/it] 

Train Epoch: 10 [30720/31158 (99%)]	Loss: 2.953014


 88%|████████▊ | 8.822463768115501/10 [21:13<02:49, 144.32s/it]

Epoch [11/10], Accuracy: 0.1238


In [45]:
# 测试模型
model.load_state_dict(torch.load('best_model_accuracy.pt'))

model.eval()  # 设置模型为评估模式
model.to(device)
with torch.no_grad():
    correct = 0
    total = 0
    for features, labels in test_loader:
        features=features.to(device)
        labels=labels.to(device)
        features = features.view(features.size(0), -1)
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f'Accuracy of the model on the test set: {100 * correct / total} %')


Accuracy of the model on the test set: 9.544679814950085 %
